In [2]:
import pandas as pd

In [3]:
#read in the dataset desired
df = pd.read_csv('diabetic_data.csv')

In [4]:
#just look at top few obs
df.head(6)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),?,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30


In [5]:
#Simplify dataframe using melt to identify patients readmission by time in hospital
df.melt(id_vars=['time_in_hospital'], value_vars=['readmitted'])

,time_in_hospital,variable,value
0,1,readmitted,NO
1,3,readmitted,>30
2,2,readmitted,NO
3,2,readmitted,NO
4,1,readmitted,NO
...,...,...,...
101761,3,readmitted,>30
101762,5,readmitted,NO
101763,1,readmitted,NO
101764,10,readmitted,NO


In [36]:
#create a pivot on the readmitted column without any aggregation - using a unique key encounter_id - no duplication
pivot_df = df.pivot(index='encounter_id', columns='readmitted', values='time_in_hospital')
print(pivot_df)

readmitted    <30  >30    NO
encounter_id                
12522         NaN  NaN  13.0
15738         NaN  NaN  12.0
16680         NaN  NaN   1.0
28236         NaN  9.0   NaN
35754         NaN  3.0   NaN
...           ...  ...   ...
443847548     NaN  3.0   NaN
443847782     NaN  NaN   5.0
443854148     NaN  NaN   1.0
443857166     NaN  NaN  10.0
443867222     NaN  NaN   6.0

[101766 rows x 3 columns]


In [32]:
#get a pivot table of age by race on time in hospital average aggregation
pivot_table_df = df.pivot_table(index='age', values='time_in_hospital', columns='race', aggfunc={'time_in_hospital': 'mean'})
pivot_table_df.columns = [f'Mean time_in_hospital ({col})' for col in pivot_table_df.columns]

In [33]:
print(pivot_table_df)

          Mean time_in_hospital (?)  Mean time_in_hospital (AfricanAmerican)  \
age                                                                            
[0-10)                     2.000000                                 2.812500   
[10-20)                    1.888889                                 3.100478   
[20-30)                    3.434783                                 3.957407   
[30-40)                    3.644737                                 3.922322   
[40-50)                    3.763636                                 4.348403   
[50-60)                    3.975069                                 4.272426   
[60-70)                    4.163636                                 4.647684   
[70-80)                    4.395659                                 4.819444   
[80-90)                    4.989924                                 4.997248   
[90-100)                   5.188406                                 4.808696   

          Mean time_in_hospital (Asian)

In [19]:
#creating a groupby object to do some calculations on numeric vars with aggregation
means_time_hosp = df['time_in_hospital'].groupby([df['age'], df['race'], df['readmitted']]).mean()

In [20]:
means_time_hosp

age       race             readmitted
[0-10)    ?                NO            2.000000
          AfricanAmerican  >30           3.200000
                           NO            2.636364
          Asian            NO            2.500000
          Caucasian        <30           2.666667
                                           ...   
[90-100)  Hispanic         <30           7.000000
                           >30           6.166667
                           NO            5.000000
          Other            >30           4.500000
                           NO            3.875000
Name: time_in_hospital, Length: 162, dtype: float64

In [21]:
#unstacking them
means_time_hosp.unstack()

readmitted                     <30       >30        NO
age      race                                         
[0-10)   ?                     NaN       NaN  2.000000
         AfricanAmerican       NaN  3.200000  2.636364
         Asian                 NaN       NaN  2.500000
         Caucasian        2.666667  2.857143  2.437500
         Hispanic              NaN       NaN  2.500000
         Other                 NaN       NaN  3.000000
[10-20)  ?                     NaN  1.500000  2.000000
         AfricanAmerican  3.461538  3.115942  3.055118
         Asian                 NaN       NaN  2.000000
         Caucasian        3.730769  4.127517  2.779468
         Hispanic              NaN  2.666667  2.750000
         Other            3.000000  3.000000  3.000000
[20-30)  ?                2.500000  3.500000  3.529412
         AfricanAmerican  3.676471  4.301205  3.833333
         Asian                 NaN       NaN  2.833333
         Caucasian        3.444444  3.636364  3.176238
         Hispanic         8.000000  3.529412  3.377778
         Other            1.000000  3.363636  3.666667
[30-40)  ?                1.000000  4.583333  3.548387
         AfricanAmerican  4.528169  4.277487  3.605150
         Asian                 NaN  2.000000  3.818182
         Caucasian        3.957198  4.094840  3.518308
         Hispanic         5.166667  4.282609  3.448276
         Other            3.636364  2.500000  3.808511
[40-50)  ?                4.222222  4.150000  3.617284
         AfricanAmerican  5.210210  4.631966  3.978044
         Asian            3.000000  4.200000  2.794872
         Caucasian        4.233871  4.223005  3.683821
         Hispanic         4.439024  4.311927  3.534759
         Other            4.000000  5.421053  3.472222
[50-60)  ?                4.181818  4.610390  3.770992
         AfricanAmerican  5.113801  4.504383  3.975526
         Asian            6.111111  5.451613  3.240964
         Caucasian        4.553754  4.320019  3.862064
         Hispanic         4.714286  4.145299  3.674419
         Other            5.083333  4.807692  3.589372
[60-70)  ?                5.242424  4.160714  4.062857
         AfricanAmerican  4.771134  4.770916  4.538153
         Asian            4.294118  4.761905  3.653465
         Caucasian        4.661126  4.445438  4.179795
         Hispanic         4.714286  4.474286  3.775194
         Other            5.565217  4.751880  4.251064
[70-80)  ?                4.529412  4.517857  4.323684
         AfricanAmerican  5.425110  4.768374  4.717122
         Asian            3.333333  4.195652  4.420561
         Caucasian        4.945418  4.532804  4.498682
         Hispanic         4.446809  4.069767  4.357143
         Other            4.914286  4.485437  4.553191
[80-90)  ?                4.800000  4.895238  5.070248
         AfricanAmerican  4.971698  4.864780  5.089783
         Asian            4.111111  3.720000  4.500000
         Caucasian        4.993782  4.702552  4.800509
         Hispanic         4.695652  4.725000  4.717949
         Other            3.800000  4.979167  4.333333
[90-100) ?                5.250000  5.142857  5.191489
         AfricanAmerican  4.914286  4.112360  5.072398
         Asian                 NaN  2.000000  5.750000
         Caucasian        4.688213  4.543605  4.837700
         Hispanic         7.000000  6.166667  5.000000
         Other                 NaN  4.500000  3.875000

In [25]:
#create a slimmed down df for this data with just the columns interested in
columns_to_keep = ['time_in_hospital', 'age', 'race', 'gender', 'readmitted']
slimmed_df = df[columns_to_keep]
print(slimmed_df)

        time_in_hospital      age             race  gender readmitted
0                      1   [0-10)        Caucasian  Female         NO
1                      3  [10-20)        Caucasian  Female        >30
2                      2  [20-30)  AfricanAmerican  Female         NO
3                      2  [30-40)        Caucasian    Male         NO
4                      1  [40-50)        Caucasian    Male         NO
...                  ...      ...              ...     ...        ...
101761                 3  [70-80)  AfricanAmerican    Male        >30
101762                 5  [80-90)  AfricanAmerican  Female         NO
101763                 1  [70-80)        Caucasian    Male         NO
101764                10  [80-90)        Caucasian  Female         NO
101765                 6  [70-80)        Caucasian    Male         NO

[101766 rows x 5 columns]


In [30]:
#do some iterating over the data with groupby
for (k1,k2), group in slimmed_df.groupby(['age','race']):
    print((k1,k2))
    print(group)


('[0-10)', '?')
       time_in_hospital     age race  gender readmitted
32389                 2  [0-10)    ?  Female         NO
('[0-10)', 'AfricanAmerican')
       time_in_hospital     age             race  gender readmitted
1579                  2  [0-10)  AfricanAmerican  Female         NO
2935                  8  [0-10)  AfricanAmerican    Male         NO
4455                  3  [0-10)  AfricanAmerican  Female        >30
6636                  2  [0-10)  AfricanAmerican    Male         NO
7363                  1  [0-10)  AfricanAmerican    Male         NO
10789                 2  [0-10)  AfricanAmerican  Female         NO
11752                 2  [0-10)  AfricanAmerican    Male         NO
18783                 2  [0-10)  AfricanAmerican    Male         NO
20466                 3  [0-10)  AfricanAmerican    Male         NO
22734                 2  [0-10)  AfricanAmerican  Female         NO
27151                 2  [0-10)  AfricanAmerican    Male        >30
35377                 2  [

In [31]:
#using groupby and aggregation on slimmed dataset
slimmed_df.groupby(['age', 'race', 'readmitted'])[['time_in_hospital']].mean()


time_in_hospital
age      race            readmitted                  
[0-10)   ?               NO                  2.000000
         AfricanAmerican >30                 3.200000
                         NO                  2.636364
         Asian           NO                  2.500000
         Caucasian       <30                 2.666667
...                                               ...
[90-100) Hispanic        <30                 7.000000
                         >30                 6.166667
                         NO                  5.000000
         Other           >30                 4.500000
                         NO                  3.875000

[162 rows x 1 columns]